In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Travel Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Travel Agent


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path="./data/부산명소 국문 정보.csv")

# 데이터 로드
docs = loader.load()

print(len(docs))
print(docs[0])

154
page_content='﻿콘텐츠ID: 255
콘텐츠명: 흰여울문화마을
구군: 영도구
위도: 35.07885
경도: 129.04402
여행지: 흰여울문화마을
제목: 가파른 절벽 끝에 흰여울문화마을
부제목: 흰여울길에서 만난 느림의 미학
주소: 부산광역시 영도구 흰여울길
연락처: 051-419-4067
홈페이지: http://www.ydculture.com/huinnyeoulculturetown/
교통정보: 도시철도 1호선 남포역 6번 출구 → 7, 71, 508 버스 환승 → 영선동 백련사 하차
버스 7,71,508 영선동 백련사 하차
주차 절영해안산책로 입구 공영 주차장 (유료)
운영일: 
휴무일: 
운영 및 시간: 매일
이용요금: 무료
편의시설: 장애인 주차장(절영해안산책로 공영주차장, 유료)
이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222164810529_ttiel
썸네일이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222164810529_thumbL
상세내용: 절영해안산책로 가파른 담벼락 위로 독특한 마을 풍경이 보인다. 해안가 절벽 끝에 바다를 따라 난 좁은 골목길 안쪽으로 작은 집들이 다닥다닥 붙어 있다. 피난민들의 애잔한 삶이 시작된 곳이자 현재는 마을주민과 함께하는 문화마을공동체 흰여울문화마을이 자리한 곳이다.
부산의 대표적인 원도심 흰여울길은 봉래산 기슭에서 굽이쳐 내리는 물줄기가 마치 흰 눈이 내리는 모습과 비슷하다 하여 이름 지어졌다. 2011년 12월, 낡은 가옥을 리모델링하면서 현재는 영도의 생활을 느낄 수 있는 독창적인 문화예술마을로 거듭났다. 

영화 ＜변호인＞을 보지 못한 사람도 그 영화의 명대사는 외울 것이다. 진우네 집 담벼락에 크게 쓰여 있기 때문이다. 많은 여행자의 영도 여행 인증샷 1호는 언제나 진우네 담벼락이었다. 흰여울길은 모두 14개의 골목길로 이어져 있고 골목골목 아기자

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def load_and_split_csv(file_path, chunk_size=1000, chunk_overlap=0):
    # CSV 로더 생성 및 데이터 로드
    loader = CSVLoader(file_path=file_path)
    docs = loader.load()
    
    # 텍스트 분할기 생성
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    
    split_docs = []
    for doc in docs:
        content = doc.page_content
        metadata = doc.metadata
        
        # '상세내용:' 이전과 이후의 텍스트 분리
        parts = content.split('상세내용:', 1)
        main_content = parts[0].strip()
        detailed_content = parts[1].strip() if len(parts) > 1 else ""
        
        if detailed_content:
            splits = text_splitter.split_text(detailed_content)
            for i, split in enumerate(splits):
                # 모든 필드를 포함하면서 상세내용만 분할
                split_content = f"{main_content}\n상세내용: {split}"
                split_docs.append(Document(
                    page_content=split_content,
                    metadata={**metadata, 'split_id': i, 'total_splits': len(splits)}
                ))
        else:
            # 상세내용이 없으면 원본 문서 그대로 추가
            split_docs.append(doc)
    
    return split_docs

# 사용 예시
file_path = "./data/부산명소 국문 정보.csv"

split_docs = load_and_split_csv(file_path)

print(f"원본 문서 수: {len(CSVLoader(file_path=file_path).load())}")
print(f"분할된 문서 수: {len(split_docs)}")
print(f"\n첫 번째 분할 문서의 내용:\n{split_docs[3].page_content}")
print(f"\n첫 번째 분할 문서의 메타데이터:\n{split_docs[3].metadata}")

원본 문서 수: 154
분할된 문서 수: 287

첫 번째 분할 문서의 내용:
﻿콘텐츠ID: 257
콘텐츠명: 국립해양박물관
구군: 영도구
위도: 35.078728
경도: 129.08018
여행지: 국립해양박물관
제목: 해양 문화의 꽃 국립해양박물관
부제목: 해양, 어디까지 알고 있니?
주소: 부산광역시 영도구 해양로301번길 45
연락처: 051-309-1900
홈페이지: https://www.mmk.or.kr/
교통정보: 도시철도 1호선 남포역 6번 출구 → 186, 66번 버스 환승 → 국립해양박물관 하차
주차 국립해양박물관 주차장 (유료)
운영일: 
휴무일: 매주 월요일
운영 및 시간: 국립해양박물관 
-화 ~ 금 09:00~18:00(입장 17:30분까지) 
-토 09:00~21:00(5~8월) / 09:00~19:00(9월~4월) 
-일 09:00~19:00 
-설날‧추석 익일, 어린이날, 광복절 09:00~21:00 

해양도서관
 -화~일요일 10:00~17:30
- 휴관일: 박물관 휴관일 및 법정공휴일
이용요금: 관람료 무료(단, 4D영상관, 유료특별전시 제외)
편의시설: 물품보관소(1, 2층), 휠체어 무료대여, 점자 블록, 점자안내판, 안내견 동반출입, 엘리베이터, 장애인 주차장, 장애인 화장실, 4D영상관 내 휠체어석
이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222175627506_ttiel
썸네일이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222175627506_thumbL
상세내용: 부산하면 바다, 바다하면 부산이라고들 하지만 우리는 대한민국 해양에 대해 얼마나 알고 있을까?
세상의 모든 바다 이야기가 시작되는 국립해양박물관이 과거와 현재 그리고 미래의 바다로 안내한다.
부산 앞 바다를 배경으로 웅장한 외관을 자랑하는 건물이 눈에 띈다. 물방울이 튀어 오르는 모양을 형상화한 외관은 마치 은하계를

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

# 단계 5: Dense Retriever 생성
# 벡터DB에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [6]:
from langchain_core.prompts import PromptTemplate

travel_planner_template = """
당신은 여행 일정 생성 AI입니다. 다음 지침에 따라 작업을 수행하세요:

시스템 인스트럭션: 여행 일정 생성 AI

1. 사용자 정보 수집:
사용자로부터 여행 취향과 관련된 정보를 수집합니다. 예를 들어, 선호하는 여행 테마, 활동, 음식, 이동 수단 등을 확인합니다.

2. 데이터 세트 필터링:
수집된 사용자 정보를 바탕으로, 제공된 부산 관련 데이터 세트(예: 관광 명소, 맛집, 테마 여행 정보 등)를 필터링하여 사용자의 여행 성향에 맞는 장소와 활동을 선택합니다.

3. 맞춤형 여행 일정 생성:
필터링된 데이터를 바탕으로, 사용자의 여행 기간과 일정에 맞는 구체적이고 체계적인 일정을 생성합니다.
일정에는 추천 방문지, 식사 장소, 이동 경로 및 시간 배분 등이 포함되어야 합니다.
또한, 일정 별 근처에 있는 지하철 역 정보를 상황에 맞게 제공해야 합니다.

4. 사용자 피드백 반영:
생성된 일정에 대해 사용자가 피드백을 제공할 경우, 일정을 수정하여 더욱 맞춤화된 계획을 제공합니다.

5. 일정 제공 및 안내:
최종 확정된 여행 일정을 사용자에게 제공하고, 일정의 각 단계에 대한 간단한 설명과 이유를 함께 제시합니다.
또한, 가게나 명소에 대한 정보를 자세히 제공해야 합니다.

6. 주어진 정보만 사용:
Based on the given context, answer the question. Only use the information from the provided context for the answer.

다음은 부산의 음식에 대한 정보입니다:
{sights_info}

이 정보를 바탕으로 체계적인 여행 일정을 작성해서 답해 주세요.

다음은 예시입니다:

예시 1:
사용자: 2024년 8월 13일부터 1박 2일 간 부산 여행을 가려고 해. 나는 해산물 음식을 좋아하고, 유명한 식당 위주로 여행을 다니고 싶어.

부산 1박 2일 유명 식당 중심 여행 일정 안내

첫째 날 (2024년 8월 13일)
- 11:00: 부산역 도착
- 12:00: 점심 - 부산역 근처 유명한 식당 '만드리곤드레밥'
- 13:30: 부산역에서 택시를 이용하여 해운대로 이동
- 14:00: 관광 - 해운대 해수욕장
- 18:00: 저녁 - 해운대 유명한 식당 '사미헌'
- 20:00: 숙소 체크인 및 휴식

둘째 날 (2024년 8월 14일)
- 09:00: 아침 - 호텔 조식 또는 근처 빵집에서 아침 식사
- 10:00: 점심 - 부산 중심지 근처 유명한 식당 '부흥식당'
- 12:00: 가볍게 산책이나 쇼핑을 즐기며 시간 보내기
- 15:00: 기념품 구매를 위해 부산 중심지로 이동
- 18:00: 저녁 - 부산 중심지 유명한 식당 '부다면옥'
- 19:30: 부산역으로 이동
- 20:00: 기차 승차

유명 식당 및 관광지
- 첫째 날 점심: 만드리곤드레밥 (부산역 근처)
- 첫째 날 저녁: 사미헌 (해운대)
- 둘째 날 점심: 부흥식당 (부산 중심지)
- 둘째 날 저녁: 부다면옥 (부산 중심지)

이동 경로
- 부산역 -> 만드리곤드레밥: 택시 또는 대중교통 이용
- 만드리곤드레밥 -> 해운대: 택시 이용
- 해운대 -> 사미헌: 택시 이용
- 사미헌 -> 숙소: 택시 이용
- 숙소 -> 부흥식당: 택시 또는 대중교통 이용
- 부흥식당 -> 부다면옥: 택시 이용
- 부다면옥 -> 부산역: 택시 또는 대중교통 이용

부산 여행을 즐기시고 즐거운 시간 보내시길 바랍니다! 여행 일정이나 추가 정보가 필요하시면 언제든지 물어보세요.

이제 사용자의 요청을 처리해 주세요:

사용자 요청: {user_request}

단계별로 생각해봅시다:
1. 사용자의 여행 기간과 선호도를 파악합니다.
2. 사용자가 원하는 특별한 장소나 음식이 있는지 확인합니다.
3. 사용자가 방문을 원하지 않는 장소가 있는지 확인합니다.
4. 제공된 정보를 바탕으로 적절한 식당과 관광지를 선택합니다.
5. 선택한 장소들을 고려하여 효율적인 동선을 계획합니다.
6. 각 날짜와 시간대별로 상세한 일정을 작성합니다.
7. 이동 방법과 주변 지하철역 정보를 포함시킵니다.
8. 선택한 식당과 관광지에 대한 간단한 설명을 추가합니다.

이제 위의 단계를 따라 사용자를 위한 맞춤형 여행 계획을 작성해 주세요.

### 부산 여행 일정 안내

[여기에 일정을 작성해 주세요]

"""

prompt = PromptTemplate.from_template(travel_planner_template)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2)

# RAG 체인 생성
rag_chain = (
    {
        "sights_info": retriever,
        "user_request": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
from langchain_teddynote.messages import stream_response

user_request = """
저는 2024년 12월 17일부터 1박 2일 동안 부산 여행을 계획하고 있어. 
나는 해산물 음식을 싫어하고, 유명한 식당과 명소를 중심으로 여행을 하고 싶어.

우선, 여행 기간은 1박 2일, 즉 2일이야. 
유명한 식당과 명소를 위주로 방문하기를 원해.
위치 간 동선을 고려해서 일정은 짜.
여행 분위기는 활동적이었으면 좋겠어.

근데, 최근에 영도에 방문한 적이 있어서 영도가 일정에 포함되지 않았으면 좋겠어.
12월 17일 11시에 부산역에 도착하고, 다음 날 저녁 20시에 부산역에서 기차를 타고 안산으로 돌아갈거야.
그리고 부산에서 유명한 이재모 피자를 먹고 싶어.

호텔의 위치는 서면입니다.
각 단계에서 방문할 식당과 장소를 구체적으로 말해줘.
"""

answer = rag_chain.stream(user_request)
stream_response(answer)

### 부산 1박 2일 여행 일정 안내

**첫째 날 (2024년 12월 17일)**

- **11:00**: 부산역 도착
  - **이동**: 도보로 이동 가능

- **12:00**: 점심 - 이재모 피자 (부산역 근처)
  - **설명**: 부산에서 유명한 피자 맛집으로, 다양한 토핑과 신선한 재료로 만든 피자가 인기입니다.

- **13:30**: 부산박물관 방문
  - **이동**: 도시철도 2호선 부산역에서 대연역까지 이동 후 도보 12분
  - **설명**: 부산의 역사와 문화를 한눈에 볼 수 있는 박물관으로, 다양한 유물과 전시가 있습니다.

- **16:00**: 삼락생태공원 방문
  - **이동**: 대연역에서 도시철도 2호선 환승 후 부산김해경전철 괘법르네시떼역 하차, 도보 16분
  - **설명**: 넓은 자연 공간에서 산책과 다양한 생태계를 경험할 수 있는 곳입니다.

- **18:30**: 저녁 - 서면 근처 유명한 식당 '부산밀면'
  - **이동**: 괘법르네시떼역에서 도시철도 2호선 서면역으로 이동
  - **설명**: 부산의 대표적인 면 요리인 밀면을 맛볼 수 있는 곳입니다.

- **20:00**: 서면 호텔 체크인 및 휴식

**둘째 날 (2024년 12월 18일)**

- **09:00**: 아침 - 호텔 조식 또는 서면 근처 카페에서 아침 식사

- **10:00**: 부산 트래블라운지 방문
  - **이동**: 서면역에서 도시철도 1호선 부산역으로 이동 후 도보 2분
  - **설명**: 부산의 다양한 여행 정보를 얻을 수 있는 곳으로, 여행자들을 위한 프로그램도 제공됩니다.

- **12:00**: 점심 - 부산역 근처 '부산돼지국밥'
  - **설명**: 부산의 대표적인 음식인 돼지국밥을 맛볼 수 있는 유명한 식당입니다.

- **14:00**: 용두산공원 방문
  - **이동**: 부산역에서 도시철도 1호선 남포역 하차 후 도보 10분
  - **설명**: 부산의 전경을 한눈에 볼 수 있는 전망대와 공원이 있는 곳